### Creating feature layer for food businesses

In [45]:
import arcgis
from arcgis.gis import GIS
from arcgis.features import FeatureLayer, FeatureLayerCollection, GeoAccessor
from arcgis.features.analysis import overlay_layers

import geopandas

import pandas as pd

import psycopg2

import sys
sys.path.append('../../../')
from utils import get_config

sys.path.append('../../')
from gis_resources import san_diego_county_zips,  execute_sql, create_where_clause, read_exact_food_biz_categories, read_exact_unhealthy_food_biz_categories

In [2]:
username = get_config("arcgis","username")
password = get_config("arcgis","passkey")
gis = GIS("https://ucsdonline.maps.arcgis.com/home", username=username, password=password)

In [3]:
qry_where = create_where_clause(read_exact_food_biz_categories())
qry = "WITH store_names AS " \
            "(SELECT DISTINCT name AS dist_names " \
             "FROM ca_business " \
        f"{qry_where}, " \
            "hpf_stores AS " \
                "(SELECT DISTINCT ON (LOWER(replace(address, name || ', ', ''))) * FROM ca_business INNER JOIN store_names ON ca_business.name = store_names.dist_names " \
                "WHERE dist_names <> 'Payless ShoeSource') " \
        "SELECT zip, name as biz_name, latitude, longitude, categories " \
        "FROM hpf_stores;"


In [4]:
nourish_user = get_config("nourish_db","username")
nourish_pswd = get_config("nourish_db","passkey")

In [5]:
conn = psycopg2.connect(
    host="awesome-hw.sdsc.edu",
    database="nourish",
    user=nourish_user,
    password=nourish_pswd)

In [6]:
foodDF = pd.DataFrame(execute_sql(conn, qry),
              columns=("zip_code","biz_name", "latitude", "longitude", "categories")
              );

# drop NaN values from column 'zip_code'
foodDF = foodDF.dropna(subset=['zip_code'])

# Convert Flow to remove decimal places
foodDF['zip_code'] = foodDF['zip_code'].astype(int)
# Convert to String
foodDF['zip_code'] = foodDF['zip_code'].astype(str)


foodDF = foodDF[foodDF['zip_code'].isin(san_diego_county_zips())]


foodDF['categories'].fillna('',inplace=True)
foodDF['categories']=foodDF.categories.apply(lambda x: ';'.join(x))


foodDF

,zip_code,biz_name,latitude,longitude,categories
0,92128,Athens Market Cafe,32.9813086,-117.08000419999999,Greek restaurant;Box lunch supplier;European r...
41,92103,Car Wash,39.2225167,-121.0536212,Gas station;Car wash
45,92131,Starbucks,32.903995,-117.10054699999999,Coffee shop;Breakfast restaurant;Cafe;Coffee s...
94,92131,The Junction,32.9030158,-117.0999425,Sports bar
107,92071,Carl's Jr.,32.8381933,-116.9795769,Fast food restaurant;Hamburger restaurant
...,...,...,...,...,...
136701,92101,Car Wash,32.729561,-117.17305189999999,Car wash
136706,91950,Welldeck Recreation Center,32.6631241,-117.1183274,Cafe;Wi-Fi spot
136735,92101,Poppy's Patio,32.7336123,-117.1488674,Restaurant
136736,92105,الصومالي,32.7487541,-117.0829478,Restaurant


In [7]:
%%time
gdf = geopandas.GeoDataFrame(
    foodDF, geometry=geopandas.points_from_xy(foodDF.longitude, foodDF.latitude))

CPU times: user 18.6 ms, sys: 2.67 ms, total: 21.2 ms
Wall time: 18.9 ms


In [8]:
%%time
foodSDF = GeoAccessor.from_geodataframe(gdf, inplace=False, column_name='SHAPE')

CPU times: user 495 ms, sys: 40.3 ms, total: 535 ms
Wall time: 504 ms


In [9]:
foodSDF

,zip_code,biz_name,latitude,longitude,categories,SHAPE
0,92128,Athens Market Cafe,32.9813086,-117.08000419999999,Greek restaurant;Box lunch supplier;European r...,"{""x"": -117.08000419999999, ""y"": 32.9813086, ""s..."
41,92103,Car Wash,39.2225167,-121.0536212,Gas station;Car wash,"{""x"": -121.0536212, ""y"": 39.2225167, ""spatialR..."
45,92131,Starbucks,32.903995,-117.10054699999999,Coffee shop;Breakfast restaurant;Cafe;Coffee s...,"{""x"": -117.10054699999999, ""y"": 32.903995, ""sp..."
94,92131,The Junction,32.9030158,-117.0999425,Sports bar,"{""x"": -117.0999425, ""y"": 32.9030158, ""spatialR..."
107,92071,Carl's Jr.,32.8381933,-116.9795769,Fast food restaurant;Hamburger restaurant,"{""x"": -116.9795769, ""y"": 32.8381933, ""spatialR..."
...,...,...,...,...,...,...
136701,92101,Car Wash,32.729561,-117.17305189999999,Car wash,"{""x"": -117.17305189999999, ""y"": 32.729561, ""sp..."
136706,91950,Welldeck Recreation Center,32.6631241,-117.1183274,Cafe;Wi-Fi spot,"{""x"": -117.1183274, ""y"": 32.6631241, ""spatialR..."
136735,92101,Poppy's Patio,32.7336123,-117.1488674,Restaurant,"{""x"": -117.1488674, ""y"": 32.7336123, ""spatialR..."
136736,92105,الصومالي,32.7487541,-117.0829478,Restaurant,"{""x"": -117.0829478, ""y"": 32.7487541, ""spatialR..."


In [10]:
foodSDF.rename(columns={'categories':'biz_type'}, inplace=True)
foodSDF.drop(columns=['latitude','longitude'], inplace=True)
foodSDF

,zip_code,biz_name,biz_type,SHAPE
0,92128,Athens Market Cafe,Greek restaurant;Box lunch supplier;European r...,"{""x"": -117.08000419999999, ""y"": 32.9813086, ""s..."
41,92103,Car Wash,Gas station;Car wash,"{""x"": -121.0536212, ""y"": 39.2225167, ""spatialR..."
45,92131,Starbucks,Coffee shop;Breakfast restaurant;Cafe;Coffee s...,"{""x"": -117.10054699999999, ""y"": 32.903995, ""sp..."
94,92131,The Junction,Sports bar,"{""x"": -117.0999425, ""y"": 32.9030158, ""spatialR..."
107,92071,Carl's Jr.,Fast food restaurant;Hamburger restaurant,"{""x"": -116.9795769, ""y"": 32.8381933, ""spatialR..."
...,...,...,...,...
136701,92101,Car Wash,Car wash,"{""x"": -117.17305189999999, ""y"": 32.729561, ""sp..."
136706,91950,Welldeck Recreation Center,Cafe;Wi-Fi spot,"{""x"": -117.1183274, ""y"": 32.6631241, ""spatialR..."
136735,92101,Poppy's Patio,Restaurant,"{""x"": -117.1488674, ""y"": 32.7336123, ""spatialR..."
136736,92105,الصومالي,Restaurant,"{""x"": -117.0829478, ""y"": 32.7487541, ""spatialR..."


In [43]:
%%time
title="San Diego County Food Businesses"
items = gis.content.search(query="title:'{}' AND type:'Feature Service'".format(title))
if len(items) > 0 and items[0].owner=='akale_UCSD':
    print("Feature Layer already exists with title {}".format(title))
    sd_food_biz_layer = items[0].layers[0]
else:
    # Convert back from a SEDF into a feature layer, and publishing on AGOL
    my_new_featurelayer = foodSDF.spatial.to_featurelayer(title=title, 
                                                             gis=gis, 
                                                             folder='nourish_gis',
                                                             tags=['SanDiegoCountyFoodBusinesses'])
    sd_food_biz_layer = my_new_featurelayer.layers[0]

Feature Layer already exists with title San Diego County Food Businesses
CPU times: user 32.7 ms, sys: 20.6 ms, total: 53.3 ms
Wall time: 1.17 s


In [38]:
# Now we need to create an overlay layer to do this feature layer on block groups instead of Zip code.
sd_imperial_block_grp_lry = FeatureLayer(url="https://services1.arcgis.com/eGSDp8lpKe5izqVc/arcgis/rest/services/ad266a/FeatureServer/0")

In [44]:
sd_food_biz_layer

<FeatureLayer url:"https://services1.arcgis.com/eGSDp8lpKe5izqVc/arcgis/rest/services/ab4e14/FeatureServer/0">

In [46]:
%%time
block_grp_zip_food_biz_overlay_features = overlay_layers(input_layer=sd_imperial_block_grp_lry,
                                              overlay_layer=sd_food_biz_layer,
                                              overlay_type='Intersect',
                                              output_name='San Diego Food Business Per Block Group',
                                             )
block_grp_zip_food_biz_overlay_features

{"cost": 13.899}


CPU times: user 642 ms, sys: 231 ms, total: 873 ms
Wall time: 1min 16s


<Item title:"San_Diego_Food_Business_Per_Block_Group" type:Feature Layer Collection owner:akale_UCSD>

In [48]:
## Now that we have an overlay layer which can tell us about block group and the food businesses associated with it.
## We can calculate density of food outlets per block group FIPS.